In [1]:
# import numpy as np
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# MAKE A SIMPLE NEURAL NETWORK

# learning parameters
lr = 0.001
batch_size = 100
num_epochs = 10
disp_step = 1
model_path = "./model_data/nn_model.ckpt"
logs_path = './logs_data/example/'

# network parameters
n_input = 784
n_hidd1 = 256
n_hidd2 = 128
n_out   = 10

# inputs
X = tf.placeholder(tf.float32, [None, n_input], name='input_data')
Y = tf.placeholder(tf.float32, [None, n_out], name='output_prediction')

# Weights
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidd1]), name='w1'),
    'b1': tf.Variable(tf.random_normal([n_hidd1]), name='b1'),
    'w2': tf.Variable(tf.random_normal([n_hidd1, n_hidd2]), name='w2'),
    'b2': tf.Variable(tf.random_normal([n_hidd2]), name='b2'),
    'wout': tf.Variable(tf.random_normal([n_hidd2, n_out]), name='wout'),
    'bout': tf.Variable(tf.random_normal([n_out]), name='bout'),
}

# Model
def multilayer_perceptron(x, weights):
    lay1 = tf.add(tf.matmul(x, weights['w1']), weights['b1'])
    lay1 = tf.nn.relu(lay1)
    lay2 = tf.add(tf.matmul(lay1, weights['w2']), weights['b2'])
    lay2 = tf.nn.relu(lay2)
    lay_out = tf.add(tf.matmul(lay2, weights['wout']), weights['bout'])
    return lay_out
 
with tf.name_scope('Model'):
    prediction = multilayer_perceptron(X, weights)
    
# Cost
with tf.name_scope('Loss'):
    op_cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                                logits=prediction, labels=Y))

# Optimisation
with tf.name_scope('Optimiser'):
    op_optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(op_cost)
    # op to apply gradients
    op_grads = tf.gradients(op_cost, tf.trainable_variables())
    op_grads = list(zip(op_grads, tf.trainable_variables()))
    

# Evaluation 
with tf.name_scope('Accuracy'):
    correct = tf.equal(tf.argmax(prediction,1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# Init
init = tf.global_variables_initializer()


In [3]:
# INITIALISE MODEL "SAVER"
saver = tf.train.Saver()

# INITIALISE MODEL "SUMMARY" for Tensorboard
# cost tensor
tf.summary.scalar("loss", op_cost)
# accuracy tensor
tf.summary.scalar("accuracy", accuracy)
# gradients
for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var)
for grad, var in op_grads:
    tf.summary.histogram(var.name + '/gradient', grad)

# Merge all summaries into a single op
op_summary = tf.summary.merge_all()

INFO:tensorflow:Summary name w1:0 is illegal; using w1_0 instead.
INFO:tensorflow:Summary name b1:0 is illegal; using b1_0 instead.
INFO:tensorflow:Summary name w2:0 is illegal; using w2_0 instead.
INFO:tensorflow:Summary name b2:0 is illegal; using b2_0 instead.
INFO:tensorflow:Summary name wout:0 is illegal; using wout_0 instead.
INFO:tensorflow:Summary name bout:0 is illegal; using bout_0 instead.
INFO:tensorflow:Summary name w1:0/gradient is illegal; using w1_0/gradient instead.
INFO:tensorflow:Summary name b1:0/gradient is illegal; using b1_0/gradient instead.
INFO:tensorflow:Summary name w2:0/gradient is illegal; using w2_0/gradient instead.
INFO:tensorflow:Summary name b2:0/gradient is illegal; using b2_0/gradient instead.
INFO:tensorflow:Summary name wout:0/gradient is illegal; using wout_0/gradient instead.
INFO:tensorflow:Summary name bout:0/gradient is illegal; using bout_0/gradient instead.


In [4]:
# TRAINING
num_batches = int(mnist.train.num_examples/batch_size)

# Start Session #1
print "Starting 1st session..."

with tf.Session() as s1:
    s1.run(init)
    
    for e in range(5):
        avg_cost = 0.
        for b in range(num_batches):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, loss = s1.run([op_optimizer, op_cost], feed_dict={X: batch_xs,\
                                                                 Y: batch_ys})
            avg_cost += loss/num_batches
            
        if e%disp_step == 0:
            print "Epoch:", '%04d' % (e+1), "cost=", "{:.9f}".format(avg_cost)
        
    # DONE FIRST PART
    print "First Optimization Finished!"
    print "Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels})
    
    # SAVE CURRENT MODEL
    save_path = saver.save(s1, model_path)
    print "Model saved in: ", save_path



Starting 1st session...
Epoch: 0001 cost= 145.968365218
Epoch: 0002 cost= 29.978168872
Epoch: 0003 cost= 19.050126300
Epoch: 0004 cost= 13.778274806
Epoch: 0005 cost= 10.319705743
First Optimization Finished!
Accuracy: 0.9207
Model saved in:  ./model_data/nn_model.ckpt


In [5]:

# Start Session #2
print "Starting 2nd session..."

with tf.Session() as s1:
    s1.run(init)
    
    load_path = saver.restore(s1, model_path)
    print "Model restored from:", load_path
    
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    # resume training
    for e in range(num_epochs):
        avg_cost = 0.
        for b in range(num_batches):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, loss, summary = s1.run([op_optimizer, op_cost, op_summary], feed_dict={X: batch_xs,\
                                                                 Y: batch_ys})
            avg_cost += loss/num_batches
            
            # Write logs at every iteration
            summary_writer.add_summary(summary, e * num_batches + b)
            
            
        if e%disp_step == 0:
            print "Epoch:", '%04d' % (e+1), "cost=", "{:.9f}".format(avg_cost)
        
    # DONE SECOND PART
    print "Second Optimization Finished!"
    print "Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels})
    
    
    print "Run the command line:\n" \
          "--> tensorboard --logdir=./logs_data " \
          "\nThen open http://0.0.0.0:6006/ into your web browser"
    
    
    

Starting 2nd session...
INFO:tensorflow:Restoring parameters from ./model_data/nn_model.ckpt
Model restored from: None
Epoch: 0001 cost= 8.046402465
Epoch: 0002 cost= 6.243569126
Epoch: 0003 cost= 4.826823533
Epoch: 0004 cost= 3.708817050
Epoch: 0005 cost= 2.901680179
Epoch: 0006 cost= 2.311864388
Epoch: 0007 cost= 1.727156266
Epoch: 0008 cost= 1.365209522
Epoch: 0009 cost= 1.048879909
Epoch: 0010 cost= 0.791116597
Second Optimization Finished!
Accuracy: 0.9431
Run the command line:
--> tensorboard --logdir=./logs_data 
Then open http://0.0.0.0:6006/ into your web browser
